In [4]:
import os
from getpass import getpass

# OpenAI API Key 설정
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")

# 모델 로드
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.3, max_tokens=1000)

# PDF 파일 로드
from langchain.document_loaders import PyPDFLoader

pdf_path = "초거대언어모델연구동향.pdf"

loader = PyPDFLoader(pdf_path)
docs = loader.load()

# 문서 청크 나누기
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)

# 벡터 임베딩 생성
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vector_dim = len(embeddings.embed_query("example text"))
index = faiss.IndexFlatL2(vector_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

# 문서 추가
for split in splits:
    vector_store.add_texts([split.page_content], metadatas=[split.metadata])

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 2})

# 프롬프트 파일 선택
def select_prompt():
    prompts = {
        "1": "prompt1.txt",
        "2": "prompt2.txt",
        "3": "prompt3.txt",
    }

    print("사용할 프롬프트를 선택하세요:")
    for key, value in prompts.items():
        print(f"{key}: {value}")

    choice = input("선택: ").strip()
    return prompts.get(choice, "prompt1.txt")  # 기본값으로 prompt1.txt 사용

prompt_file = select_prompt()

# 선택한 프롬프트 파일 읽기
with open(prompt_file, "r", encoding="utf-8") as file:
    system_message = file.read()

# ChatPromptTemplate 구성
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_message_template = SystemMessagePromptTemplate.from_template(system_message)
human_message_template = HumanMessagePromptTemplate.from_template("{user_input}")

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_template, human_message_template]
)

# RAG 체인 클래스 및 디버그 기능 추가
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import LLMChain
from datetime import datetime

class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output

class ContextToText:
    def invoke(self, inputs, config=None, **kwargs):
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

contextual_prompt = LLMChain(
    prompt=chat_prompt,
    llm=model,
)

# 질문 응답 체인 구성
def get_contextual_response(query):
    # 리트리버에서 컨텍스트 가져오기
    context = retriever.get_relevant_documents(query)

    # 문서 내용을 텍스트로 변환
    context_text = "\n".join([doc.page_content for doc in context])

    # 최종 시스템 및 사용자 메시지 생성
    formatted_messages = chat_prompt.format_messages(
        context=context_text, user_input=query
    )

    # 모델 실행
    response = DebugPassThrough().invoke(model(formatted_messages))
    return response

# 결과 저장 경로
output_folder = r"C:\Users\USER\mission\mission\Results"
os.makedirs(output_folder, exist_ok=True)  # 폴더가 없으면 생성

# 질문 입력 및 처리
query = input("질문을 입력하세요: ")

# 모델로부터 답변 생성
response = get_contextual_response(query)

# 답변 출력
print("Final Response: ")
print(response.content)

# 결과 파일 저장
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")  # 현재 시각으로 파일명 생성
output_file = os.path.join(output_folder, f"response_{timestamp}.txt")

with open(output_file, "w", encoding="utf-8") as file:
    file.write(f"질문: {query}\n\n")
    file.write(f"답변:\n{response.content}\n")

print(f"답변이 다음 위치에 저장되었습니다: {output_file}")


OpenAI API key 입력: ········
사용할 프롬프트를 선택하세요:
1: prompt1.txt
2: prompt2.txt
3: prompt3.txt
선택: 3
질문을 입력하세요: 초거대 언어모델 최근 주요 연구 동향을 알려줘
Debug Output: content='질문의 의도는 최근 초거대 언어모델(Super Large Language Models) 분야에서의 주요 연구 동향을 파악하고자 하는 것으로 보입니다. 이는 연구자들이 현재의 기술 발전을 이해하고, 향후 연구 방향을 설정하는 데 도움이 될 수 있습니다. \n\n최근 초거대 언어모델에 대한 연구 동향은 다음과 같은 몇 가지 주요 주제로 요약될 수 있습니다:\n\n1. **모델의 효율성 향상**: 많은 연구자들이 모델의 크기와 성능 간의 균형을 맞추기 위해 효율적인 아키텍처를 개발하고 있습니다. 예를 들어, Sparse Attention, Mixture of Experts(MoE)와 같은 기법이 연구되고 있습니다. 이러한 기법들은 모델의 파라미터 수를 줄이면서도 성능을 유지하거나 향상시키는 데 기여하고 있습니다.\n\n2. **지속적인 학습과 적응성**: 초거대 언어모델이 새로운 데이터에 적응할 수 있도록 하는 연구가 활발히 진행되고 있습니다. 이는 모델이 훈련 후에도 지속적으로 학습할 수 있는 방법을 모색하는 것으로, 예를 들어, 메타 학습이나 온라인 학습 기법이 포함됩니다.\n\n3. **윤리적 고려사항 및 편향 문제**: 초거대 언어모델이 사회에 미치는 영향에 대한 연구가 증가하고 있습니다. 모델의 편향 문제를 해결하고, 윤리적인 AI 개발을 위한 가이드라인을 제시하는 연구가 활발히 이루어지고 있습니다.\n\n4. **멀티모달 학습**: 텍스트뿐만 아니라 이미지, 비디오 등 다양한 형태의 데이터를 처리할 수 있는 멀티모달 모델에 대한 연구가 증가하고 있습니다. 이러한 모델은 다양한 입력을 통합하여 더 풍부한 정보를 제공할 수 있습니다.\n\n5. **자원 소모와 환경적 영향**: 초거대